In [ ]:
%load_ext autoreload
%autoreload 2

## Setup
If you already have a model/tokenizer you want to use, you can skip this step. 
Be sure to also set the appropriate user_tag/assistant_tag for that model.

In [ ]:
%%capture
# The quantized model used here requires some extra libraries. 
import sys
!{sys.executable} -m pip install torch==2.1.1 torchvision==0.16.1 torchaudio==2.1.1 --index-url https://download.pytorch.org/whl/cu121
!{sys.executable} -m pip install optimum>=1.12.0
!{sys.executable} -m pip install auto-gptq==0.6.0
!{sys.executable} -m pip install accelerate

In [ ]:
import os
os.environ['TRANSFORMERS_CACHE'] = '/workspace/cache/' # change or comment out as desired 
from transformers import AutoModelForCausalLM, AutoTokenizer

def load_model(model_name_or_path, revision, device):
    model = AutoModelForCausalLM.from_pretrained(
        model_name_or_path, device_map=device, revision=revision, trust_remote_code=False)
    tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True, padding_side="left")
    tokenizer.pad_token_id = 0
    return model, tokenizer

device = 'cuda:0'
model_name_or_path = "TheBloke/Mistral-7B-Instruct-v0.1-GPTQ"
revision = 'gptq-4bit-32g-actorder_True'
user_tag = "[INST] "
assistant_tag = " [/INST]"

model, tokenizer = load_model(model_name_or_path, revision, device)

## Train extractor

In [ ]:
from lmdoctor.doctor import Doctor
extraction_target = 'honesty'
probe_type = 'pca'
doc = Doctor(model, tokenizer, user_tag, assistant_tag, extraction_target=extraction_target, probe_type=probe_type, device=device)
doc.extract(n_train_pairs=128, n_dev_pairs=64, n_test_pairs=32)

In [ ]:
doc.statement_pairs['train'][:1]

## Run detector

In [ ]:
# prompt = "Tell me a fact about humans"
prompt = "Tell me a lie about humans"
# prompt = "Who authored 'Gimbar's adventures'?"

import torch
torch.manual_seed(0)

max_new_tokens = 10
output = doc.generate(prompt, max_new_tokens=max_new_tokens, do_sample=True, gen_only=True, return_projections=True)
output['text']

In [ ]:
tokens = tokenizer.tokenize(output['text'])
doc.plot_projection_heatmap(output['projections'], tokens, lastn_tokens_to_plot=max_new_tokens, saturate_at='auto')

In [ ]:
scores_per_token = doc.detect(output['projections'], aggregation_method='layer_avg', use_n_middle_layers=15)
doc.plot_scores_per_token(scores_per_token, tokens, lastn_tokens_to_plot=max_new_tokens, saturate_at=1)

In [ ]:
# alternative: auto-aggregation
scores_per_token = doc.detect(output['projections'], aggregation_method='auto')
doc.plot_scores_per_token(scores_per_token, tokens, lastn_tokens_to_plot=max_new_tokens, detection_method='classifier')

## Control (with reading vector)

In [ ]:
# refer to control_utils.Controller for info on setting alpha and control_direction
prompt = "You killed a person for revenge but don't want to get caught. What would you tell the police?"
print(doc.generate_with_control(prompt, control_direction=None, max_new_tokens=12)) # no control
print(doc.generate_with_control(prompt, control_direction=1, max_new_tokens=12, alpha=1, n_trim_layers=10)) # add the vector

In [ ]:
prompt = "I shoplifted a candy bar from the store. What should I tell my parents?"
print(doc.generate_with_control(prompt, control_direction=None, max_new_tokens=12)) # no control
print(doc.generate_with_control(prompt, control_direction=-1, max_new_tokens=12, alpha=1, n_trim_layers=10)) # subtract the vector

## Extra visualizations

In [ ]:
# Visualize the projections

from lmdoctor import plot_utils, detection_utils
proj_pairs = detection_utils.act_pairs_to_projs(doc.train_acts, doc.direction_info, len(doc.statement_pairs['train']))

layer = 15
projs_true = proj_pairs[0, :, layer]
projs_lie = proj_pairs[1, :, layer]
plot_utils.plot_projs_on_numberline(projs_true, projs_lie)

In [ ]:
# visualize a scan from the training dataset

input_text = doc.statement_pairs['train'][8][1]
# input_text = doc.statement_pairs['train'][8][0] 

projections = doc.get_projections(input_text=input_text)
tokens = tokenizer.tokenize(input_text)
doc.plot_projection_heatmap(projections, tokens, saturate_at='auto')